<a href="https://colab.research.google.com/github/bstrain71/usn_instruction_search/blob/master/usn_instruction_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### INGEST

OPNAV Instructions: https://www.secnav.navy.mil/doni/opnav.aspx?RootFolder=%2Fdoni%2FDirectives%2F01000%20Military%20Personnel%20Support&FolderCTID=0x012000E8AF0DD9490E0547A7DE7CF736393D04&View=%7BCACF3AEF%2DAED4%2D433A%2D8CE5%2DA45245715B5C%7D

Note: This example only contains series 01-01 through 01-500 as it is just for demonstration. Here is a key to their topics:

 
01-01 General Military Personnel Records

01-100 General Recruiting Records 

01-200 Personnel Classification and Designation 

01-300 Assignment and Distribution Services  

01-400 Promotion and Advancement Programs 

01-500 Military Training and Education Services 



In [0]:
%matplotlib inline

In [2]:
# Load gdrive
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
!pip install -q tika #install tika

In [12]:
from tika import parser

# Test code for seeing how tika works.
raw = parser.from_file('/content/gdrive/My Drive/MSDS_422/OPNAV_Instructions/1160.8B.pdf')
# raw['content'] gives the raw text of the pdf
#print(raw['content'])
raw.keys()
#raw['metadata']['resourceName'] # Calls the file name - not metadata. not every file has metadata so don't rely on it.

dict_keys(['status', 'content', 'metadata'])

In [13]:
import os
#from datetime import datetime

# Measure the time it takes to append
# all of the documents. If this ends
# up taking too long find a way to
# optimize it.
#start=datetime.now()

# The indices for these are the same so they can be zipped later.
# The .pdf filename is what is read, not the metadata, so whatever
# the file is called in the folder is what it will be listed as
# in the results.

documents = [] # Contains raw text from all .pdfs.
filenames = [] # Contains the filenames of the .pdfs

# Parse all .pdf contents and their filenames.
directory = '/content/gdrive/My Drive/MSDS_422/OPNAV_Instructions/'
for file in os.listdir(directory):
  temp = parser.from_file(directory+file)
  documents.append(temp['content'])
  filenames.append(temp['metadata']['resourceName'])

#end=datetime.now()
#print(end-start)
print(len(filenames))
print(len(documents))

165
165


### EDA and Data Cleaning

Make everything in the document lowercase and remove stopwords. The stopwords list can be modified as needed, this one comes directly from the gensim documentation and is relatively standard.

In [0]:
from collections import defaultdict
from gensim import corpora

# improved list from Stone, Denis, Kwantes (2010)
STOPWORDS = """
a about above across after afterwards again against all almost alone along already also although always am among amongst amoungst amount an and another any anyhow anyone anything anyway anywhere are around as at back be
became because become becomes becoming been before beforehand behind being below beside besides between beyond bill both bottom but by call can
cannot cant co computer con could couldnt cry de describe
detail did didn do does doesn doing don done down due during
each eg eight either eleven else elsewhere empty enough etc even ever every everyone everything everywhere except few fifteen
fify fill find fire first five for former formerly forty found four from front full further get give go
had has hasnt have he hence her here hereafter hereby herein hereupon hers herself him himself his how however hundred i ie
if in inc indeed interest into is it its itself keep last latter latterly least less ltd
just
kg km
made make many may me meanwhile might mill mine more moreover most mostly move much must my myself name namely
neither never nevertheless next nine no nobody none noone nor not nothing now nowhere of off
often on once one only onto or other others otherwise our ours ourselves out over own part per
perhaps please put rather re
quite
rather really regarding
same say see seem seemed seeming seems serious several she should show side since sincere six sixty so some somehow someone something sometime sometimes somewhere still such system take ten
than that the their them themselves then thence there thereafter thereby therefore therein thereupon these they thick thin third this those though three through throughout thru thus to together too top toward towards twelve twenty two un under
until up unless upon us used using
various very very via
was we well were what whatever when whence whenever where whereafter whereas whereby wherein whereupon wherever whether which while whither who whoever whole whom whose why will with within without would yet you
your yours yourself yourselves
"""

# remove common words and tokenize
stoplist = set(STOPWORDS.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

# Building the corpus.
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

### Modeling

Build the model to measure similarity between documents. Here we will use Latent Semantic Analysis (LSI)

In [0]:
from gensim import models


# "Indexing by Latent Semantic Analysis" <http://www.cs.bham.ac.uk/~pxt/IDA/lsa_ind.pdf>
# Latent Semantic Indexing <https://en.wikipedia.org/wiki/Latent_semantic_indexing>


# change num_topics to change singular values
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=100)

Now suppose a user typed in the query [whatever doc = 'user stuff']. We would like to sort the corpus documents in decreasing order of relevance to this query. Unlike modern search engines, here we only concentrate on a single aspect of possible similarities—on apparent semantic relatedness of their texts (words). No hyperlinks, no random-walk static ranks, just a semantic extension over the boolean keyword match:

In [0]:
# Here the 'doc' object is the user's search term.
# vec_bow cleans the user's search term
# vec_lsi is a list of tuples which are the similarities,

# This is only the boolean similarity!
# This is the same as doing Ctrl+F and counting
# the number of hits.

doc1 = "gmt"
vec_bow1 = dictionary.doc2bow(doc1.lower().split())
vec_lsi1 = lsi[vec_bow1]  # convert the query to LSI space

doc2 = "lateral transfer"
vec_bow2 = dictionary.doc2bow(doc2.lower().split())
vec_lsi2 = lsi[vec_bow2]  # convert the query to LSI space

doc3 = "warfare pin"
vec_bow3 = dictionary.doc2bow(doc3.lower().split())
vec_lsi3 = lsi[vec_bow3]  # convert the query to LSI space

doc4 = "ldo package"
vec_bow4 = dictionary.doc2bow(doc4.lower().split())
vec_lsi4 = lsi[vec_bow4]  # convert the query to LSI space

doc5 = "career intermission program"
vec_bow5 = dictionary.doc2bow(doc5.lower().split())
vec_lsi5 = lsi[vec_bow5]  # convert the query to LSI space

In addition, we will be considering `cosine similarity <http://en.wikipedia.org/wiki/Cosine_similarity>`_
to determine the similarity of two vectors. Cosine similarity is a standard measure
in Vector Space Modeling, but wherever the vectors represent probability distributions,
`different similarity measures <http://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence#Symmetrised_divergence>`_
may be more appropriate.

Initializing query structures
++++++++++++++++++++++++++++++++

To prepare for similarity queries, we need to enter all documents which we want
to compare against subsequent queries.



In [47]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi[corpus])  # transform corpus to LSI space and index it

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


<div class="alert alert-danger"><h4>Warning</h4><p>The class :class:`similarities.MatrixSimilarity` is only appropriate when the whole
  set of vectors fits into memory. For example, a corpus of one million documents
  would require 2GB of RAM in a 256-dimensional LSI space, when used with this class.

  Without 2GB of free RAM, you would need to use the :class:`similarities.Similarity` class.
  This class operates in fixed memory, by splitting the index across multiple files on disk, called shards.
  It uses :class:`similarities.MatrixSimilarity` and :class:`similarities.SparseMatrixSimilarity` internally,
  so it is still fast, although slightly more complex.</p></div>

Index persistency is handled via the standard :func:`save` and :func:`load` functions:



In [48]:
index.save('/tmp/deerwester.index')
index = similarities.MatrixSimilarity.load('/tmp/deerwester.index')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


This is true for all similarity indexing classes (:class:`similarities.Similarity`,
:class:`similarities.MatrixSimilarity` and :class:`similarities.SparseMatrixSimilarity`).
Also in the following, `index` can be an object of any of these. When in doubt,
use :class:`similarities.Similarity`, as it is the most scalable version, and it also
supports adding more documents to the index later.

Performing queries
++++++++++++++++++

To obtain similarities of our query document against the nine indexed documents:



In [49]:
# index is the similarity index, or what we are checking against (corpus in the lsi space)
# vec_lsi is our query in the lsi space
# the output is the similarity tuples

sims1 = index[vec_lsi1]
similarity_list1 = list(zip(sims1, filenames))
#print(similarity_list)
print(doc1)
print(sorted(similarity_list1, reverse = True))

sims2 = index[vec_lsi2]
similarity_list2 = list(zip(sims2, filenames))
#print(similarity_list)
print(doc2)
print(sorted(similarity_list2, reverse = True))

sims3 = index[vec_lsi3]
similarity_list3 = list(zip(sims3, filenames))
#print(similarity_list)
print(doc3)
print(sorted(similarity_list3, reverse = True))

sims4 = index[vec_lsi4]
similarity_list4 = list(zip(sims4, filenames))
#print(similarity_list)
print(doc4)
print(sorted(similarity_list4, reverse = True))

sims5 = index[vec_lsi5]
similarity_list5 = list(zip(sims5, filenames))
#print(similarity_list)
print(doc5)
print(sorted(similarity_list5, reverse = True))

gmt
[(0.41427985, '1500.22H.pdf'), (0.34469885, '1700.16B.pdf'), (0.28215128, '1650.35A.pdf'), (0.26691204, '1650.24D.pdf'), (0.24847959, '1520.43B.pdf'), (0.24369234, '1531.6D.pdf'), (0.23602673, '1520.42A CH-1.pdf'), (0.23544645, '1520.38A.pdf'), (0.21633464, '1070.2C.pdf'), (0.21162046, '1120.13B.pdf'), (0.20615228, '1531.2C.pdf'), (0.20241559, '1700.13C CH-1.pdf'), (0.19610748, '1650.25C.pdf'), (0.19495277, '1000.1N.pdf'), (0.1946071, '1700.1164.pdf'), (0.19320172, '1650.36.pdf'), (0.1854843, '1542.7E.pdf'), (0.18440205, '1541.5.pdf'), (0.18313679, '1520.44.pdf'), (0.17446479, '1306.4A.pdf'), (0.16459817, '1300.18B.pdf'), (0.15844858, '1738.1A.pdf'), (0.15579681, '1160.6C.pdf'), (0.15559986, '1650.33A.pdf'), (0.15123741, '1120.3B.pdf'), (0.15030287, '1754.4A.pdf'), (0.14931083, '1754.7A OPNAV.pdf'), (0.14310408, '1770.2B.pdf'), (0.13924798, '1520.18J.pdf'), (0.12545127, '1110.1B.pdf'), (0.12274447, '1754.8.pdf'), (0.12038446, '1220.2B.pdf'), (0.11949172, '1430.4C.pdf'), (0.1186976,

### Conclusion

This model could be very useful in production. There is a huge corpus of instructions that USN personnel have to sort through in order to get information - an easy way to query this corpus to find relevant instructions given a topic would save everyone time and energy. There is no objective metric with which to measure 'accuracy' but the searches have consistently returned sane and sensible results during operator testing.